In [ ]:
# import os
# os.environ["ORCHESTRA_DEPLOYMENT_TYPE"] = "remote"
# os.environ["DEV_MODE"] = "True"
# os.environ["TEST_EXTERNAL_REGISTRY"] = "k3d-registry.localhost:5800"

In [ ]:
# isort: off
# stdlib
import random
import os
import sys

# syft absolute
import syft as sy
from syft.util.util import (
    find_base_dir_with_tox_ini,
    get_caller_file_path,
    is_interpreter_jupyter,
)


def add_helper_path_to_python_path() -> None:
    current_path = "."

    # jupyter uses "." which resolves to the notebook
    if not is_interpreter_jupyter():
        # python uses the file which has from syft import test_settings in it
        import_path = get_caller_file_path()
        if import_path:
            current_path = import_path

    base_dir = find_base_dir_with_tox_ini(current_path)
    notebook_helper_path = os.path.join(
        base_dir, "notebooks/scenarios/bigquery/upgradability/0.9.1_helpers"
    )
    sys.path.append(notebook_helper_path)


add_helper_path_to_python_path()
from syft.service.job.job_stash import Job

# third party
from email_helpers import get_email_server
from job_helpers import approve_by_running
from job_helpers import get_job_emails
from job_helpers import get_request_for_job_info
# isort: on

In [ ]:
ADMIN_EMAIL, ADMIN_PW = "admin2@bigquery.org", "bqpw2"

# Start server & login

In [ ]:
server = sy.orchestra.launch(
    name="bigquery-high-migrations",
    dev_mode=True,
    server_side_type="high",
    port="8080",
    n_consumers=1,  # How many workers to be spawned
    create_producer=True,  # Can produce more workers
)

In [ ]:
high_client = sy.login(
    url="http://localhost:8080", email=ADMIN_EMAIL, password=ADMIN_PW
)

In [ ]:
email_server, smtp_server = get_email_server()

# Review requests

In [ ]:
# third party
from email_helpers import load_users
from job_helpers import load_jobs
from job_helpers import save_jobs

In [ ]:
high_client.requests.get_all_pending()

In [ ]:
users = load_users(high_client)
jobs_data = load_jobs(users, high_client)
all_requests = high_client.requests
submitted_jobs_data = [job for job in jobs_data if job.is_submitted]
n_emails_per_job_user = {
    k: len(v)
    for k, v in get_job_emails(submitted_jobs_data, high_client, email_server).items()
}

In [ ]:
# TODO we should record whether it was approved or deposited
# and test doing both in either order as there might be a bug when
# force overwriting
# also changing deny to approve and back again

# Run or deny

In [ ]:
submitted_jobs_data_should_succeed = [
    j for j in submitted_jobs_data if j.should_succeed
]
submitted_jobs_data_should_fail = [
    j for j in submitted_jobs_data if not j.should_succeed
]

In [ ]:
for job in submitted_jobs_data_should_succeed:
    request = get_request_for_job_info(all_requests, job)
    if random.randrange(2):
        choice = "approved with deposit_result"
        response = approve_by_running(request)
        assert isinstance(response, Job)
    else:
        choice = "approved"
        response = request.approve()
        assert isinstance(response, sy.SyftSuccess)
    print(f"Job {job.func_name} should succeed: {job.should_succeed} and was {choice}")
    job.admin_reviewed = True

In [ ]:
for job in submitted_jobs_data_should_fail:
    request = get_request_for_job_info(all_requests, job)
    response = request.deny(
        reason=f"Your request {job.func_name} looks wrong, try again."
    )
    assert isinstance(response, sy.SyftSuccess)
    assert not job.should_succeed
    job.admin_reviewed = True

# Verify that users have new emails

In [ ]:
new_n_emails_per_job_user = {
    k: len(v)
    for k, v in get_job_emails(submitted_jobs_data, high_client, email_server).items()
}

In [ ]:
# for user_email, new_count in new_n_emails_per_job_user.items():
#     old_count = n_emails_per_job_user[user_email]
#     assert new_count > old_count

# Save state

In [ ]:
save_jobs(jobs_data)

In [ ]:
high_client.requests.get_all_approved()

In [ ]:
high_client.requests.get_all_rejected()

# Cleanup

In [ ]:
smtp_server.stop()

In [ ]:
server.land()